### Notebook for preparing the Human Cell Atlas (human heart leucocytes) dataset as a reference dataset for seed label transfer to mouse ACM heart (merged Pkp2+Ttn dataset) using `scANVI` 

#### Environment: scANVI

- **Developed by:** Alexandra Cirnu
- **Modified by:** Alexandra Cirnu
- **Würzburg Institute for Systems Immunology & Julius-Maximilian-Universität Würzburg**
- **Date of creation:** 240308
- **Date of modification:** 240308

Healthy human heart leucocytes (from Human cell atlas) was used to generate a reference for seed labelling. 


#### Import required modules

In [1]:
import anndata
import warnings
import numpy as np
import scanpy as sc
import pandas as pd

In [2]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

-----
anndata     0.10.5.post1
scanpy      1.9.8
-----
PIL                 10.2.0
asttokens           NA
colorama            0.4.6
comm                0.2.1
cycler              0.12.1
cython_runtime      NA
dateutil            2.9.0
debugpy             1.8.1
decorator           5.1.1
exceptiongroup      1.2.0
executing           2.0.1
h5py                3.10.0
ipykernel           6.29.3
ipywidgets          8.1.2
jedi                0.19.1
joblib              1.3.2
kiwisolver          1.4.5
llvmlite            0.42.0
matplotlib          3.8.3
mpl_toolkits        NA
natsort             8.4.0
numba               0.59.0
numpy               1.26.4
packaging           23.2
pandas              2.2.1
parso               0.8.3
platformdirs        4.2.0
prompt_toolkit      3.0.43
psutil              5.9.8
pure_eval           0.2.2
pydev_ipython       NA
pydevconsole        NA
pydevd              2.9.5
pydevd_file_utils   NA
pydevd_plugins      NA
pydevd_tracing      NA
pygments            2.17.

In [3]:
def X_is_raw(adata): return np.array_equal(adata.X.sum(axis=0).astype(int), adata.X.sum(axis=0))

### Read in Human Cell Atlas Cardiac Leucocytes data set

Data downloaded from [here](https://cellgeni.cog.sanger.ac.uk/heartcellatlas/data/hca_heart_immune_download.h5ad)

In [4]:
adata = sc.read_h5ad('/home/acirnu/data/ACM_cardiac_leuco/Reference_data/hca_heart_immune_download.h5ad')
adata

AnnData object with n_obs × n_vars = 40868 × 33538
    obs: 'NRP', 'age_group', 'cell_source', 'cell_states', 'donor', 'gender', 'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'region', 'sample', 'scrublet_score', 'type', 'version', 'scNym', 'scNym_confidence'
    uns: 'cell_states_colors', 'scNym_colors', 'scNym_probabilities'
    obsm: 'X_pca', 'X_scnym', 'X_umap'

In [5]:
del(adata.obs['age_group'])
del(adata.obs['percent_mito'])
del(adata.obs['percent_ribo'])
del(adata.obs['scrublet_score'])
del(adata.obs['type'])
del(adata.obs['version'])

adata.obs

,NRP,cell_source,cell_states,donor,gender,n_counts,n_genes,region,sample,scNym,scNym_confidence
AAAGTGAAGTCGGCCT-1-H0015_apex,No,Harvard-Nuclei,CD4+T_cytox,H5,Female,724.717285,588,AX,H0015_apex,CD4+T_cell,0.797180
AAATGGAAGGTCCCTG-1-H0015_apex,No,Harvard-Nuclei,CD4+T_cytox,H5,Female,668.059509,515,AX,H0015_apex,CD4+T_cell,0.999248
AAATGGAGTTGTCTAG-1-H0015_apex,No,Harvard-Nuclei,doublets,H5,Female,670.216309,504,AX,H0015_apex,NK,0.680673
AACAACCGTAATTGGA-1-H0015_apex,No,Harvard-Nuclei,DOCK4+MØ1,H5,Female,730.082947,578,AX,H0015_apex,CD14+Monocyte,0.538159
AAGACTCTCAGGACGA-1-H0015_apex,No,Harvard-Nuclei,Mast,H5,Female,612.323425,428,AX,H0015_apex,Mast,0.990977
...,...,...,...,...,...,...,...,...,...,...,...
TTTGATCGTGTCATGT-1-HCAHeart8102862,Yes,Sanger-CD45,CD4+T_cytox,D11,Female,631.149170,715,AX,HCAHeart8102862,CD8+T_cell,0.756579
TTTGATCGTTCTCCTG-1-HCAHeart8102862,Yes,Sanger-CD45,LYVE1+MØ1,D11,Female,819.040100,2526,AX,HCAHeart8102862,CD8+T_cell,0.269561
TTTGGAGGTCGCTCGA-1-HCAHeart8102862,Yes,Sanger-CD45,MØ_AgP,D11,Female,757.455505,1350,AX,HCAHeart8102862,M3,0.585436
TTTGGTTTCAGTGTTG-1-HCAHeart8102862,Yes,Sanger-CD45,LYVE1+MØ3,D11,Female,815.372131,2507,AX,HCAHeart8102862,MØ,0.968681


In [6]:
X_is_raw(adata)

True

In [7]:
adata.obs['cell_states'].cat.categories

Index(['NK', 'CD16+Mo', 'DOCK4+MØ1', 'CD4+T_cytox', 'LYVE1+MØ1', 'CD8+T_tem',
       'CD8+T_cytox', 'LYVE1+MØ2', 'LYVE1+MØ3', 'CD14+Mo', 'Mo_pi',
       'DOCK4+MØ2', 'Mast', 'NKT', 'MØ_mod', 'MØ_AgP', 'B_cells', 'CD4+T_tem',
       'DC', 'doublets', 'NØ', 'IL17RA+Mo'],
      dtype='object')

In [8]:
adata.obs['seed_labels'] = adata.obs['cell_states'].copy()

In [10]:
trans_from = [['CD4+T_cytox', 'CD4+T_tem'],                             #CD4
              ['CD8+T_tem', 'CD8+T_cytox'],                             #CD8
              ['NK'],                                                   #NK
              ['NKT'],                                                  #NKT
              ['CD16+Mo', 'CD14+Mo', 'Mo_pi', 'IL17RA+Mo'],             #Monocytes
              ['DOCK4+MØ1','LYVE1+MØ1', 'LYVE1+MØ2', 'LYVE1+MØ3', 'DOCK4+MØ2', 'MØ_mod', 'MØ_AgP'],                #Macrophages
              ['B_cells'],                                              #B cells
              ['Mast'],                                                 #Mast_cells
              ['DC'],                                                   #DC
              ['NØ'],                                                   #Neutrophils
              ['doublets']]                                             #Doublets

trans_to = ['CD4+T', 'CD8+T', 'NK', 'NKT','Monocytes', 'Macrophages', 'B_cells', 'Mast_cells', 'DC', 'Neutrophils', 'Doublets']

adata.obs['seed_labels'] = [str(i) for i in adata.obs['cell_states']]
for leiden,celltype in zip(trans_from, trans_to):
    for leiden_from in leiden:
        adata.obs['seed_labels'][adata.obs['seed_labels'] == leiden_from] = celltype

/tmp/ipykernel_1059153/3830905514.py:18: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  adata.obs['seed_labels'][adata.obs['seed_labels'] == leiden_from] = celltype
/tmp/ipykernel_1059153/3830905514.py:18: SettingWithCopyWarning: 
A value is 

In [11]:
adata.obs['seed_labels'].value_counts()

seed_labels
Macrophages    14519
Monocytes       6845
CD8+T           5957
CD4+T           4160
NK              3628
Mast_cells      1543
NKT             1463
B_cells         1195
DC               814
Doublets         623
Neutrophils      121
Name: count, dtype: int64

In [12]:
adata.obs

,NRP,cell_source,cell_states,donor,gender,n_counts,n_genes,region,sample,scNym,scNym_confidence,seed_labels
AAAGTGAAGTCGGCCT-1-H0015_apex,No,Harvard-Nuclei,CD4+T_cytox,H5,Female,724.717285,588,AX,H0015_apex,CD4+T_cell,0.797180,CD4+T
AAATGGAAGGTCCCTG-1-H0015_apex,No,Harvard-Nuclei,CD4+T_cytox,H5,Female,668.059509,515,AX,H0015_apex,CD4+T_cell,0.999248,CD4+T
AAATGGAGTTGTCTAG-1-H0015_apex,No,Harvard-Nuclei,doublets,H5,Female,670.216309,504,AX,H0015_apex,NK,0.680673,Doublets
AACAACCGTAATTGGA-1-H0015_apex,No,Harvard-Nuclei,DOCK4+MØ1,H5,Female,730.082947,578,AX,H0015_apex,CD14+Monocyte,0.538159,Macrophages
AAGACTCTCAGGACGA-1-H0015_apex,No,Harvard-Nuclei,Mast,H5,Female,612.323425,428,AX,H0015_apex,Mast,0.990977,Mast_cells
...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGATCGTGTCATGT-1-HCAHeart8102862,Yes,Sanger-CD45,CD4+T_cytox,D11,Female,631.149170,715,AX,HCAHeart8102862,CD8+T_cell,0.756579,CD4+T
TTTGATCGTTCTCCTG-1-HCAHeart8102862,Yes,Sanger-CD45,LYVE1+MØ1,D11,Female,819.040100,2526,AX,HCAHeart8102862,CD8+T_cell,0.269561,Macrophages
TTTGGAGGTCGCTCGA-1-HCAHeart8102862,Yes,Sanger-CD45,MØ_AgP,D11,Female,757.455505,1350,AX,HCAHeart8102862,M3,0.585436,Macrophages
TTTGGTTTCAGTGTTG-1-HCAHeart8102862,Yes,Sanger-CD45,LYVE1+MØ3,D11,Female,815.372131,2507,AX,HCAHeart8102862,MØ,0.968681,Macrophages


#### Define 15000 highly variable genes and save as a reference file

In [13]:
adata_raw = adata.copy()
adata.layers['counts'] = adata.X.copy()

In [14]:
adata15000 = adata.copy()

sc.pp.highly_variable_genes(
    adata15000,
    flavor = "seurat_v3",
    n_top_genes = 15000,
    layer = "counts",
    batch_key = "donor",
    subset = True,
    span = 1
    )

adata15000

If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
--> added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)


AnnData object with n_obs × n_vars = 40868 × 15000
    obs: 'NRP', 'cell_source', 'cell_states', 'donor', 'gender', 'n_counts', 'n_genes', 'region', 'sample', 'scNym', 'scNym_confidence', 'seed_labels'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'cell_states_colors', 'scNym_colors', 'scNym_probabilities', 'hvg'
    obsm: 'X_pca', 'X_scnym', 'X_umap'
    layers: 'counts'

In [15]:
adata15000.write('/home/acirnu/data/ACM_cardiac_leuco/Reference_data/HCA_all_celltypes_healthy_reference_HVG15000_ac240308.raw.h5ad')  